In [1]:
import requests
from Config import ConsumerKey as key

In [2]:
# define our endpoint
endpoint = r'https://api.tdameritrade.com/v1/marketdata/chains'

In [3]:
# definte our payload
payload = {'apikey':key,
           'symbol':'QQQ',
           'strikeCount':'30',
           'fromDate':'2022-04-01'}

In [4]:
# make a request
content = requests.get(url = endpoint, params = payload)

In [5]:
# convert it to a dictionary
data = content.json()

In [6]:
import pandas as pd

In [7]:
optionsChain = pd.json_normalize(data) 

In [8]:
categories = ['putCall','symbol', 'volatility','delta','gamma',
              'theta','vega','rho','totalVolume','openInterest','strikePrice']

In [9]:
selected = pd.DataFrame(columns = categories, dtype = object)

In [10]:
for i in range(len(optionsChain.transpose())-12):
    for j in categories:
        # iterate and put values into 'selected' dataframe
        selected.at[i, j] = optionsChain.iat[0, i+12][0].get(j)

In [11]:
selected

,putCall,symbol,volatility,delta,gamma,theta,vega,rho,openInterest,strikePrice
0,PUT,QQQ_050622P298,NaN,NaN,NaN,NaN,-999,NaN,1593,298
1,PUT,QQQ_050622P299,NaN,NaN,NaN,NaN,-999,NaN,2279,299
2,PUT,QQQ_050622P300,NaN,NaN,NaN,NaN,-999,NaN,16951,300
3,PUT,QQQ_050622P301,NaN,NaN,NaN,NaN,-999,NaN,4721,301
4,PUT,QQQ_050622P302,45.134,-0.09,0.017,-0.386,0.035,-0.001,3552,302
...,...,...,...,...,...,...,...,...,...,...
1735,CALL,QQQ_011924C365,24.252,0.434,0.004,-0.04,1.658,1.872,491,365
1736,CALL,QQQ_011924C370,24.062,0.415,0.004,-0.039,1.64,1.801,2659,370
1737,CALL,QQQ_011924C375,23.878,0.396,0.004,-0.038,1.619,1.729,435,375
1738,CALL,QQQ_011924C380,23.691,0.377,0.004,-0.037,1.594,1.656,1228,380


In [12]:
'''cycle through all dates; add them to set; create new dfs with set; place relevant items into relevant dfs'''

'cycle through all dates; add them to set; create new dfs with set; place relevant items into relevant dfs'

In [13]:
%%time
# create set for use with dataFrame creation loop (name of dfs = date portion of 'symbol')
dateSet = []
for i in selected['symbol']:
#     print(selected['symbol'][4:10])
#     print(i[4:10])
    if i[4:10] not in dateSet:
        a = i[4:10]
        dateSet.append(a)
print(dateSet)

['050622', '050922', '051122', '051322', '051622', '051822', '052022', '052322', '052522', '052722', '053122', '060122', '060322', '061022', '061722', '062422', '063022', '071522', '081922', '091622', '093022', '121622', '123022', '012023', '031723', '033123', '061623', '121523', '011924']
Wall time: 1 ms


In [14]:
# creating dataframes via loop with names = items from dateSet above
d = {}
for name in dateSet:
    d[name] = pd.DataFrame(columns = categories, dtype = object)

In [15]:
%%time
# loop through all dateSet dates, then loop through all rows in selected(dataframe),
#  check if date from dateSet is within symbol from current row of selected(dataframe),
#  if date from dateSet is within symbol from current row of selected(dataframe) then add the 
#  current row of selected(dataframe) into the dateSet dates' dataframe (dataframes are housed
#  within the 'd' dictionary which is a dictionary of dataframes created for each date)
for dates in dateSet:
#     print(i)
    counter = 0
    for symbol in selected['symbol']:
        
        if dates in symbol:
#             print(j)
#             print(d[i])
            d[dates].loc[len(d[dates].index)] = selected.loc[counter]
        counter += 1

Wall time: 8.27 s


In [16]:
'''Run script once a day before open; store all tables into a dictionary named after the current date'''

'Run script once a day before open; store all tables into a dictionary named after the current date'

In [17]:
from datetime import date

In [18]:
currentDay = date.today()

In [19]:
today = str(currentDay.month)+str(currentDay.day)+str(currentDay.year)

In [20]:
# Checking if single digit date and adding zeros accordingly; formatting entire date properly
if len(str(currentDay.day)) == 1:
    correctedDay = str(currentDay.day).zfill(2)
#     print(correctedDay)
else:
    correctedDay = currentDay.day
#     print(correctedDay)
if len(str(currentDay.month)) == 1:
    correctedMonth = str(currentDay.month).zfill(2)
#     print(correctedMonth)
else:
    correctedMonth = currentDay.month
#     print(correctedmonth)
    
today = str(correctedMonth)+str(correctedDay)+str(currentDay.year)
today

'05052022'

In [21]:
import pickle

In [22]:
try:
    with open('mypickle.pickle', 'rb') as f:
        bigDictionary = pickle.load(f)
except Exception as e:
    print(e)

In [23]:
# bigDictionary = {}

In [24]:
# Iteratively adding dictionaries per day
bigDictionary[today] = d

In [25]:
'''begin pickle process to commit each day's addition to a running file'''

"begin pickle process to commit each day's addition to a running file"

In [26]:
with open('mypickle.pickle', 'wb') as f:
    pickle.dump(bigDictionary, f, protocol=pickle.HIGHEST_PROTOCOL)